# Linear Ramping Difference Check

By Matthew Davis

This code performs the simulations for my masters thesis. This is for the section related to diagonal dispatch targets.

Here we download electricity price data for several regions. Then we run a linear optimiser to find the optimal charge/discharge production schedule for a hypothetical battery. We do this twice for each series of prices:

* *stepped*: within each time interval, the power input/output is constant. At the boundary between trading periods, the power level changes instantaneously.
* *diagonal*: Power level is always continuous. It is a sequence of diagonal 'dot-to-dot ramps'

The most important part of this code is the function `dispatch_mode` in section `Dispatch Mode`. That's the point of the whole paper.

This notebook can be run on any normal laptop. The computational requirements are not large, in terms of memory. However it takes several hours to run. (Set `debug=True` later to speed it up while modifying the notebook.) This includes downloading the data, which will be skipped automatically on subsequent runs.

Install dependencies with `pip install -r requirements.txt`. Then modify `data_location` to point somewhere appropriate for your device.

For dataframes, I use [Polars](https://docs.pola.rs/) instead of Pandas, just because I think the interface is nicer, and that's what I'm used to these days. If you're not familiar with Polars, but you know Pandas, you'll still be able to read and understand most of the Polars code.

The code runs simulations for Europe, but I don't include the results in my thesis. It turns out Europe's grid rules are quite complicated, and this diagonal/linear dispatch constraint does not apply. But I'm leaving the code in because it might be handy to have later.

## Imports and Setup

In [93]:
import datetime as dt
import os
import urllib.request
from multiprocessing import Pool, cpu_count
from random import randrange
from time import time
from zipfile import ZipFile
from itertools import product
from urllib3.util.retry import Retry



from requests import Session
from requests.adapters import HTTPAdapter
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.subplots as sp
import polars as pl
from nemosis import dynamic_data_compiler
from pyomo.environ import (
    AbstractModel,
    Binary,
    Boolean,
    ConcreteModel,
    Constraint,
    NonNegativeReals,
    Objective,
    Param,
    Reals,
    Set,
    SolverFactory,
    Var,
    maximize,
)

In [94]:
# where to store the raw price data
# About 10GB
data_location = "/home/matthew/Data/ramping/"

# where to store the tables and graphs for the paper
results_location = "results/"

In [95]:
aus_data_location = "/home/matthew/Data/nemosis/"
eu_data_location = os.path.join(data_location, "Europe")
miso_location = os.path.join(data_location, "USA", "MISO")

In [96]:
start_date = dt.date(2024, 1, 1)
end_date = dt.date(2024, 12, 31)

In [97]:
# basic unit conversions
ms_per_s = 1000
s_per_min = 60
min_per_h = 60
ms_per_h = ms_per_s * s_per_min * min_per_h

gw_per_mw = 1 / 1000

## Download Data

In [98]:
for d in [data_location, aus_data_location, eu_data_location, miso_location, results_location]:
    os.makedirs(d, exist_ok=True)

### Australia

We use [nemosis](https://github.com/UNSW-CEEM/NEMOSIS/) to download the [`DISPATCHPRICE` MMS table](https://nemweb.com.au/Reports/Current/MMSDataModelReport/Electricity/Electricity%20Data%20Model%20Report_files/Electricity%20Data%20Model%20Report_toc.htm) from AEMO's [nemweb](https://www.nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/2025/MMSDM_2025_03/MMSDM_Historical_Data_SQLLoader/DATA/). 

Note that:

* We delete `INTERVENTION == 1` columns. Those are academic counterfactuals when AEMO manually overrides instructions for specific generators. `INTERVENTION == 0` is what generators get paid, so they're the 'real' prices we're concerned about.
* Timestamps in AEMO data always refer to the *end* of the interval, not the start. This doesn't really matter for our purposes.
* `RRP` means price ($/MWh)

In [99]:
# Prices in Australia are 5-minute granularity
interval_min_aus = 5

In [ ]:
nemosis_dt_fmt = "%Y/%m/%d %H:%M:%S"

aus_df_pd = dynamic_data_compiler(
    start_date.strftime(nemosis_dt_fmt),
    end_date.strftime(nemosis_dt_fmt),
    "DISPATCHPRICE",
    aus_data_location,
)

INFO: Compiling data for table DISPATCHPRICE


In [ ]:
aus_df = (
    pl.from_pandas(aus_df_pd)
    .filter(pl.col("INTERVENTION") == 0)
    .select(time="SETTLEMENTDATE", region="REGIONID", price="RRP")
)

aus_df.head()

### Europe

We download European price data from [Ember](https://ember-energy.org/data/european-wholesale-electricity-price-data/). Note that this is day ahead data, not intra-day, because intraday prices are not public. (You have to pay a lot.)

In [ ]:
url = "https://storage.googleapis.com/emb-prod-bkt-publicdata/public-downloads/price/outputs/european_wholesale_electricity_price_data_hourly.zip"
zip_path = os.path.join(eu_data_location, "raw.zip")

In [ ]:
# don't re-download if already present
if not os.path.exists(zip_path):
    urllib.request.urlretrieve(url, zip_path)

Now read in the CSV.

Inside the zip, it looks like:

```
Country,ISO3 Code,Datetime (UTC),Datetime (Local),Price (EUR/MWhe)
France,FRA,2015-01-01 00:00:00,2015-01-01 01:00:00,36.56
```

In [ ]:
with ZipFile(zip_path, "r") as zf:
    with zf.open("all_countries.csv", "r") as csv_f:
        eu_df = pl.read_csv(csv_f, try_parse_dates=True)

In [ ]:
eu_df = eu_df.select(
    region="Country",
    time="Datetime (UTC)",
    price="Price (EUR/MWhe)",
)

In [ ]:
# which countries do we have data for?
(
    eu_df
    .select("region")
    .unique()
    .sort("region")
)

In [ ]:
# this data is hourly
interval_min_eu = min_per_h

### MISO

Download from [here](https://www.misoenergy.org/markets-and-operations/real-time--market-data/market-reports/#nt=%2FMarketReportType%3AHistorical%20LMP%2FMarketReportName%3AReal-Time%205-Min%20ExAnte%20LMPs%20(xlsx)&t=10&p=0&s=MarketReportPublished&sd=desc).

Market Reports > Historical LMP > Real-Time 5-Min ExAnte LMPs (xlsx).
One file for each day.



In [ ]:

s = Session()

# aggressive retries. MISO's server is very slow
retry = Retry(
    total=8,
    read=8,
    connect=8,
    backoff_factor=1.5,
    raise_on_status=False
)
adapter = HTTPAdapter(max_retries=retry)
s.mount("http://", adapter)
s.mount("https://", adapter)


d = start_date
dates = [start_date]
while d <= end_date:
    dates.append(d)
    d += dt.timedelta(days=1)

for d in tqdm(dates):
    url = f"https://docs.misoenergy.org/marketreports/{d.strftime('%Y%m%d')}_5min_exante_lmp.xlsx"
    local_xls_path = os.path.join(miso_location, url.split("/")[-1])
    local_pq_path = os.path.join(miso_location, url.split("/")[-1].rpartition('.')[0] + ".parquet")

    # download
    if not os.path.exists(local_xls_path):  # don't re-download
        
        try:
            with s.get(url, stream=True) as r:
                r.raise_for_status()
                with open(local_xls_path, "wb") as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:  # filter out keep-alive chunks
                            f.write(chunk)
        except Exception:
            if os.path.exists(local_xls_path):
                os.remove(local_xls_path)
            raise

    # convert to parquet
    if not os.path.exists(local_pq_path):
        (
            pl.read_excel(
                local_xls_path, 
                read_options = {"header_row": 3}, 
            )
            .lazy()
            .cast({"RT Ex-Ante LMP": pl.Float64}) # in case some are empty
            .group_by(pl.col("Time (EST)").alias("time"))
            .agg(
                pl.col("RT Ex-Ante LMP").mean().alias("price")
            )
            .sink_parquet(local_pq_path)
        )

    if pl.read_parquet_schema(local_pq_path)['price'] == pl.String():
        print(f"{local_pq_path.split('/')[-1]}: {pl.read_parquet_schema(local_pq_path)}")
        break
    

In [ ]:
# load the data from disk
miso_df = pl.read_parquet(f"{miso_location}/*.parquet")
miso_df

In [ ]:
# this is 5 minute data
interval_min_miso = 5

### Sample

Construct some trivial data to test the code gives reasonable optimal strategies.

In [ ]:
start_datetime = dt.datetime(start_date.year, start_date.month, start_date.day)

In [ ]:
prices = [10, 100, 10, 10, 100, 100, 10, 10, 10, 100] + [
    randrange(0, 100) for _ in range(20)
]
sample_prices = pl.DataFrame(
    {
        "price": prices,
        "time": pl.datetime_range(
            start=start_datetime,
            end=start_datetime + dt.timedelta(hours=len(prices) - 1),
            interval="1h",
            eager=True,
        ),
    }
).with_columns(region=pl.lit("fake"))


sample_prices

## Combine and Prepare Price Data

We want one Polars dataframe, with columns:

* `region`, e.g. "France" or "NSW1" (part of Aus)
* `price`
* `period` (integer starting at 0 for each region)
* `time` (datetime)

In [ ]:
price_df = (
    pl.concat(
        [
            sample_prices,
            aus_df,
            eu_df,
            miso_df.with_columns(pl.lit('MISO').alias('region')),
        ],
        how="diagonal_relaxed",
    )
    .filter(pl.col("time").is_between(start_date, end_date))
    .filter(pl.col("price").is_not_null())
    .sort("region", "time")
    .group_by("region")
    .map_groups(lambda group: group.with_row_index("period"))
)

In [ ]:
# restrict to only these regions
# otherwise it takes too long
regions = ["SA1", "QLD1", "NSW1", "VIC1", "TAS1", "MISO"]
#regions = ["fake"]

price_df = price_df.filter(pl.col("region").is_in(regions))

In [ ]:
region_metadata_df = (
    price_df.select("region")
    .unique()
    .with_columns(
        pl.when(pl.col("region").is_in(aus_df["region"].implode()))
        .then(interval_min_aus)
        .when(pl.col("region") == "MISO")
        .then(interval_min_miso)
        .otherwise(interval_min_eu)
        .alias("interval_min")
    )
    .with_columns((pl.col("interval_min") / min_per_h).alias("h_per_interval"))
)
# convert to dict for easy lookup later
region_metadata = {
    r: {"h_per_interval": h, "interval_min": i}
    for (r, i, h) in zip(
        region_metadata_df.to_dict(as_series=False)["region"],
        region_metadata_df.to_dict(as_series=False)["interval_min"],
        region_metadata_df.to_dict(as_series=False)["h_per_interval"],
    )
}
region_metadata

## Optimising

### Simple Simulation Parameters

In [ ]:
# how big is the battery?
battery_max_power = 1  # MW
battery_depth_h = 2  # hours
battery_depth_mwh = battery_max_power * battery_depth_h

# starting conditions, chosen arbitrarily
initial_charge = battery_depth_mwh / 2
initial_power = battery_max_power / 2

round_trip_efficiency = 0.8

In [ ]:
model = AbstractModel()

In [ ]:
model.period = Set(ordered=True, name="period")
model.prices = Param(model.period, domain=Reals, name="price")
model.h_per_interval = Param(domain=Reals, name="h_per_interval")

### Variables and Constraints

*Parameters* are things we know at the start of the computation. e.g. prices. (We assume perfect forsight of prices). 
*Variables* are things we decide (i.e. how much power to produce).

In practice generators/batteries choose bids, which are a set of power levels conditional on price. We're abstracting from that and pretending firms choose production levels (i.e. power) directly, because we're assuming perfect forsight of prices.

*Power* is the instantaneous *flow* of the good (MW, megawatts). *Energy* is a stock (MWh, megawatt hours), which accumulates as the *charge* on the battery.

To account for round-trip losses, we have a separate charge and discharge variable. Then we have constraints to limit them so we don't charge and discharge at the same time.

In [ ]:
model.directions = Set(
    initialize=["charge", "discharge"], ordered=True, name="direction"
)

# MW power
# at the end of each interval
# separated by charge/discharge (gross, not net)
model.power_gross_end = Var(
    model.period,
    model.directions,
    domain=NonNegativeReals,
    bounds=[0, battery_max_power],
    name="power_gross_end",
)

# MW power
# at the end of each interval
# netted (sum of charge/discharge together)
# positive means supplied to grid (discharge)
model.power_net_end = Var(
    model.period,
    domain=Reals,
    bounds=[-battery_max_power, battery_max_power],
    name="power_net_end",
)

# MW power
# averaged across the interval
# separated by charge/discharge (gross)
model.power_gross_avg = Var(
    model.period,
    model.directions,
    domain=NonNegativeReals,
    bounds=[0, battery_max_power],
    name="power_gross_avg",
)

# MW supplied/consumed from grid
# averaged over whole period
# charge + discharge combined (net)
# positive means supplied
model.power_net_avg = Var(
    model.period,
    domain=Reals,
    bounds=[-battery_max_power, battery_max_power],
    name="power_net_avg",
)

In [ ]:
# net power is the sum of charge and discharge power
# arbitrarily define positive power as being discharging, negative as charge
# so positive power means positive revenue (when prices are positive)
def power_netting_end(model, p):
    return (
        model.power_net_end[p]
        == model.power_gross_end[p, "discharge"]
        - model.power_gross_end[p, "charge"]
    )


model.power_netting_end_constraint = Constraint(
    model.period, rule=power_netting_end
)

In [ ]:
# netting of averages
def power_netting_avg(model, p):
    return (
        model.power_net_avg[p]
        == model.power_gross_avg[p, "discharge"] - model.power_gross_avg[p, "charge"]
    )

# redundant constraint, so omit it
#model.power_netting_avg_constraint = Constraint(model.period, rule=power_netting_avg)

Given the <100% efficiency, the optimal solution during negative prices is to simultaneously charge and discharge, to get paid to waste power. However this is not physically possible.

Although if last period we were charging, and this period we want to discharge, due to the linear ramping, we will have to continue charging for the first few seconds/minutes, charging more slowly, then eventually flip to discharging. So within one interval we do both.

So instead I'll use a lazy solution, to just cap the sum of the two at the maximum of each individual charge/discharge. This is equivilent to saying that the battery can charge for x% of an interval, and discharge for the remainder of the interval.

In [ ]:
def only_one_direction(model, p):
    return (
        model.power_gross_end[p, "charge"]
        + model.power_gross_end[p, "discharge"]
        <= battery_max_power
    )

model.only_one_direction_constr = Constraint(
    model.period, rule=only_one_direction
)

### Dispatch Mode

This is what the difference is between the two approaches. This is the most important part of this notebook. The rest of the optimiser stuff is a straightforward battery optimisation.

Here we say `step` approach is when the starting power each interval can be different to the end of the previous interval, and power is constant within each interval. Whereas `diagonal` means it must start at the same level it finished the previous period at, and then we will ramp the power level (smoothly change production level) towards the power level at the end of the interval.

```
        ----|
----|   |   |
    |   |   |
    ----|   ----
```

vs

```
-----      /\
     \    /  \
      \  /    \
       \/      \
```

We have variables for power at the _end_ of each interval. For the stepped approach, the average power during an interval equals the power at the end. For the diagonal approach, the average power during an interval is the average of the power at the end of this interval and the end of the previous interval.

In [ ]:
model.is_diagonal = Param(domain=Boolean)

def dispatch_mode(model, p):
    if model.is_diagonal:
        if p > 0:
            start_power = model.power_net_end[p - 1]
        else:
            # initial constraint
            start_power = initial_power
        end_power = model.power_net_end[p]
        return model.power_net_avg[p] == (start_power + end_power) / 2
    else:
        # constant power
        return model.power_net_avg[p] == model.power_net_end[p]


model.dispatch_mode_constr = Constraint(model.period, rule=dispatch_mode)

In [ ]:
# same again, but separate charge/discharge
# this may be a redundant constraint

def dispatch_mode_gross(model, p, d):
    if model.is_diagonal:
        if p > 0:
            start_power = model.power_gross_end[p - 1, d]
        else:
            # initial constraint
            start_power = initial_power
        end_power = model.power_gross_end[p, d]
        return model.power_gross_avg[p, d] == (start_power + end_power) / 2
    else:
        # constant power
        return model.power_gross_avg[p, d] == model.power_gross_end[p, d]


model.dispatch_mode_constr_gross = Constraint(model.period, model.directions, rule=dispatch_mode_gross)

### Charge Level

Charge level is in MWh, energy. It's a stock, not flow.
This stops us from discharging fully, repeatedly, every interval (which would maximize revenue, but is impossible)


In [ ]:
# MWh at end of the interval
model.charge_end = Var(
    model.period,
    domain=NonNegativeReals,
    bounds=[0, battery_depth_mwh],
    name="charge",
)

For linking power (flow) to charge (stock), we apply a round trip energy loss `round_trip_efficiency`. This asymmetry is why we separate charge and discharge.

In [ ]:
def charge_from_power(model, p):
    # * h_per_interval to go from MW to MWh
    if p > 1:
        start_charge = model.charge_end[p-1]
    else:
        start_charge = initial_charge

    charge_change = model.h_per_interval * (
        model.power_gross_avg[p, "charge"]
        * round_trip_efficiency
        - model.power_gross_avg[p, "discharge"]
    )
    return model.charge_end[p] == start_charge + charge_change

model.charge_from_power_constraint = Constraint(model.period, rule=charge_from_power)

### Objective

The objective is to maximize spot market revenue,
(given a sunk infrastructure investment).
The only cost is the cost of charging the battery. So this is a pure temporal arbitrage business.

In [ ]:
model.revenue = Var(model.period, domain=Reals, name="revenue")


# definition of revenue each period
def revenue_eq(model, p):
    # note that power is MW, price is $/MWh
    # so we need to scale by number of hours
    return (
        model.revenue[p]
        == model.power_net_avg[p] * model.prices[p] * model.h_per_interval
    )


model.revenue_def = Constraint(model.period, rule=revenue_eq)


# summing up revenue
# In a callable function, because we don't yet know how many time periods we will have.
def total_revenue_rule(model):
    return sum(model.revenue[p] for p in model.period)


model.revenue_total = Objective(
    expr=total_revenue_rule,
    sense=maximize,
)

### Solve

Run the solver now. This is what takes a long time.

The solver is single-threaded. I'm lazy, and not wrapping it in `multiprocessing.Pool`. Partly because if someone else wants to run this, but they use windows, they may have troubles. (multiprocessing + Jupyter + Windows don't work together). Also, to keep the memory footprint small enough for a normal lapop.

In [ ]:
# While writing code, we can speed things up by shortenning the time period.
debugging = False
if debugging:
    price_df = price_df.group_by("region").head(1000)

In [ ]:
def solve(region, is_diagonal, region_price_df):
    print(f"Computing result for {region} {is_diagonal=}")
    start_time = time()
    # strange nested dicts:
    # https://pyomo.readthedocs.io/en/stable/howto/abstract_models/data/raw_dicts.html#page-data-from-dict
    instance = model.create_instance(
        {
            None: {
                "period": {None: region_price_df["period"]},
                "prices": {
                    t: p
                    for (t, p) in zip(
                        region_price_df["period"], region_price_df["price"]
                    )
                },
                "is_diagonal": {None: is_diagonal},
                "h_per_interval": {None: region_metadata[region]["h_per_interval"]},
            }
        }
    )

    result = SolverFactory("glpk", executable="/usr/bin/glpsol").solve(instance)

    assert (
        result["Solver"][0]["Termination condition"] != "infeasible"
    ), f"infeasible for {region}"

    mid_time = time()

    # s_per_min
    print(
        f"Calculations for {region} {is_diagonal=} took {(mid_time - start_time) :.1f} s"
    )

    data = {}

    # values indexed only on time
    for col in ["revenue", "power_net_avg", "power_net_end", "charge_end"]:
        data[col] = [getattr(instance, col)[t].value for t in instance.period]

    # values indexed on time and direction
    for col in ["power_gross_avg", "power_gross_end"]:
        for d in instance.directions:
            data[col] = [getattr(instance, col)[t, d].value for t in instance.period]

    # for values indexed on time, and direction
    for col in ["power_gross_avg"]:
        for d in instance.directions:
            data[col.replace("gross", d)] = [
                getattr(instance, col)[t, d].value for t in instance.period
            ]

    df = (
        pl.DataFrame(data)
        .with_row_index(name="period")
        .with_columns(
            pl.lit(is_diagonal).alias("is_diagonal"),
        )
        .join(region_price_df, how="left", on="period")
    )


    return df #(df, instance, result)

If you're using Windows, the next cell may fail. If so, this is a problem with your installation of python, not with my code. This is a known issue with Jupyter + multiprocessing + windows. Either get the code sample at the top of [this doc page](https://docs.python.org/3/library/multiprocessing.html#module-multiprocessing) working, or rewrite the cell below this one with a for loop.

In [ ]:
def echo(x):
    return x

payload = list(range(5))
with Pool() as p:
    result = p.map(echo, payload)

assert result == payload

In [ ]:
# tasks = []
# for (region, ) in price_df.select("region").unique().iter_rows():
#     for is_diagonal in [True, False]:
#         input_df = price_df.filter(pl.col("region") == region)
#         tasks.append((region, is_diagonal, input_df))


# def _solve_unwrap(payload):
#     (region, is_diagonal, input_df) = payload
#     return solve(region, is_diagonal, input_df)


# with Pool(processes=cpu_count() - 1) as p:
#     results_iter = p.map(_solve_unwrap, tasks)
# print('calculated, now concating')
# results = pl.concat(results_iter, how="vertical_relaxed")
# 'done'

In [ ]:
# regions = [r for (r,) in price_df.select("region").unique().iter_rows()]
# regions

# def _solve_unwrap(payload):
#     (region, is_diagonal) = payload
#     input_df = price_df.filter(pl.col("region") == region)
#     return solve(region, is_diagonal, input_df)



# with Pool(processes=cpu_count() - 1) as p:
#     results_iter = p.map(_solve_unwrap, product(regions, [True, False]))
# results = pl.concat(results_iter, how="vertical_relaxed")

Multiprocessing hangs on completion. I don't know why. I guess Polars' arrow backend can't handle it.

In [ ]:
results = []

for (region,) in price_df.select("region").unique().iter_rows():
    for is_diagonal in [True, False]:
        assert isinstance(region, str), f"Bad iter_rows() call: {region}"
        #h_per_interval = region_metadata[region]["h_per_interval"]
        input_df = price_df.filter(pl.col("region") == region)
        output_df = solve(region, is_diagonal, input_df)
        results.append(output_df)
results = pl.concat(results, how="vertical_relaxed")
results

In [ ]:
results

### Analyse Results

#### Summary Stats

Start with single statistics.

For "partial cycles", we want to count how many times the battery changes direction from charge to discharge, or back. Counting partial cycles (not fully depleting/charging.)

In [ ]:
os.makedirs(results_location, exist_ok=True)

In [ ]:
results_summary = (
    results.with_columns(
        pl.when(pl.col("power_net_avg") > 0)
        .then(1)
        .when(pl.col("power_net_avg") < 0)
        .then(-1)
        .fill_null(strategy="forward")
        .alias("direction")
    )
    .with_columns(
        (pl.col("direction").shift(1).over("region") != pl.col("direction")).alias(
            "change_direction"
        )
    )
    .join(region_metadata_df, on="region")
    .group_by("region", "is_diagonal")
    .agg(
        (pl.col("power_charge_avg") * pl.col("h_per_interval"))
        .sum()
        .alias("total_charged_mwh"),
        (pl.col("power_discharge_avg") * pl.col("h_per_interval"))
        .sum()
        .alias("total_discharged_gwh"),
        pl.col("revenue").sum().alias("total_revenue"),
        pl.col("change_direction").sum().alias("direction_changes"),
    )
    .sort("region", "is_diagonal")
)
results_summary.write_csv(os.path.join(results_location, "simulation-results.csv"))

results_summary

Now let's print the table into a form I can put into Typst.

In [ ]:
attributes = [
    ("total_discharged_gwh", "Energy (GWh)", 2, gw_per_mw),
    ("total_revenue", "Profit (\\$k)", 1, 1 / 1000),
    # ("direction_changes", "Partial Cycles", 0, 1),
]

num_columns = 1 + 1 + 3 * len(attributes)

table_output = os.path.join(results_location, "simulation-results.typ")

header = f"""
#set text(size: 9.5pt)
#table(
    columns: {num_columns},
    align: horizon,
    stroke: (x: none, y: 0.4pt),
    table.cell(
        colspan: 2,
    )[Region],
"""

with open(table_output, "w") as f:

    f.write(header)

    for i, (df_col, table_col, dp, scale) in enumerate(attributes):
        f.write(f"  table.cell(colspan: 3,)[{table_col}],\n")
        # if i < len(attributes) - 1: # if not last
        #    f.write("  [],\n")

    f.write("  [Name], [Interval],\n")
    for i, (df_col, table_col, dp, scale) in enumerate(attributes):
        f.write(f"  [#text(blue)[Step]], [#text(red)[Diagonal]], [Difference],\n")
        # if i < len(attributes) - 1: # if not last
        #    f.write("  [],\n")

    regions = [r for (r,) in results.select("region").unique().iter_rows()]
    table_row = 3
    for j, region in enumerate(regions):
        if region.lower() != "fake":
            interval_min = region_metadata[region]["interval_min"]
            if interval_min == min_per_h:
                interval_s = f"1h"
            else:
                interval_s = f"{interval_min}m"
            f.write(f"  [{region}], [{interval_s}],\n")
            for i, (df_col, table_col, dp, scale) in enumerate(attributes):
                stepped = (
                    results_summary.filter(pl.col("region") == region)
                    .filter(pl.col("is_diagonal").not_())
                    .select(df_col)
                    .item()
                ) * scale
                diagonal = (
                    results_summary.filter(pl.col("region") == region)
                    .filter(pl.col("is_diagonal"))
                    .select(df_col)
                    .item()
                ) * scale
                diff = (stepped - diagonal) / stepped
                f.write(
                    f"     [{stepped:.{dp}f}], [{diagonal:.{dp}f}], [{diff * 100:.1f} %],\n"
                )

            if j < len(regions) - 1:  # if not last
                f.write(f"    table.hline(y: {table_row}, stroke: none),\n")
            table_row += 1
    f.write(")\n")

Now print it as a LaTeX table.

In [ ]:
num_columns = 2 + 3 * len(attributes)

table_output = os.path.join(results_location, "simulation-results.tex")

col_spec = 'll' + 'r' * 3 * len(attributes)

with open(table_output, "w") as f:

    f.write("\\begin{tabular}{@{}" + col_spec + "@{}}\n")
    f.write("  \\toprule\n")
    f.write("  \\multicolumn{2}{c}{Region}\n")

    for (df_col, table_col, dp, scale) in attributes:
        f.write("    & \\multicolumn{3}{c}{" + table_col + "}\n")
    f.write("    \\\\")

    f.write("    Name & Interval\n")
    for (df_col, table_col, dp, scale) in attributes:
        f.write("   & Stepped & Diagonal & Difference\n")
    f.write("    \\\\ \\midrule\n")

    regions = [r for (r,) in results.select("region").unique().iter_rows()]
    for region in regions:
        if region.lower() != "fake":
            interval_min = region_metadata[region]["interval_min"]
            if interval_min == min_per_h:
                interval_str = f"1h"
            else:
                interval_str = f"{interval_min}m"

            f.write(f"    {region} & {interval_str}\n")
            
            for df_col, table_col, dp, scale in attributes:
                stepped = (
                    results_summary.filter(pl.col("region") == region)
                    .filter(pl.col("is_diagonal").not_())
                    .select(df_col)
                    .item()
                ) * scale
                diagonal = (
                    results_summary.filter(pl.col("region") == region)
                    .filter(pl.col("is_diagonal"))
                    .select(df_col)
                    .item()
                ) * scale
                diff = (stepped - diagonal) / stepped
                f.write(
                    f"      & {stepped:,.{dp}f} & {diagonal:,.{dp}f} & {diff * 100:.1f} \\% \n"
                )
            f.write("    \\\\ \n")
    f.write("  \\bottomrule")
    f.write("\\end{tabular}")

#### Plot

Now find the period of time where the results differ the most, and then plot a time-series window showing the difference. For each region.

In [ ]:
window_size = 12
interesting_periods = (
    results.pivot("is_diagonal", index=["period", "region"], values="power_net_avg")
    .rename({"true": "diagonal", "false": "step"})
    .with_columns((pl.col("step") - pl.col("diagonal")).pow(2).alias("difference"))
    .sort("period")
    .cast({"period": pl.Int32})
    .group_by_dynamic("period", every=f"{window_size}i", group_by="region")
    .agg(pl.col("difference").sum())
    .sort("difference", descending=True)
    .group_by("region")
    .head(1)
    .select("region", "period")
)
interesting_periods

In [ ]:
for region, period in interesting_periods.iter_rows():

    fig = sp.make_subplots(rows=2, cols=1, shared_xaxes=True)

    plot_df = results.filter(pl.col("region") == region).filter(
        pl.col("period").is_between(
            period - (window_size // 2), period + 2 * window_size
        )
    )

    diagonal_df = plot_df.filter(pl.col("is_diagonal") == True)
    step_df = plot_df.filter(pl.col("is_diagonal") == False)
    price_plot_df = diagonal_df  # choose either, to deduplicate

    fig.add_trace(
        go.Scatter(
            x=diagonal_df["period"],
            y=diagonal_df["power_net_end"],
            mode="lines",
            marker={"color": "blue"},
            name="diagonal",
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=step_df["period"],
            y=step_df["power_net_end"],
            mode="lines",
            line={"color": "red", "shape": "vh"},
            name="stepped",
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=price_plot_df["time"],
            y=price_plot_df["price"],
            mode="lines",
            line={"shape": "vh"},
            name="Prices",
        ),
        row=2,
        col=1,
    )

    #fig.update_layout(title=f"Simulation Results - {region}")
    fig.update_yaxes(title_text="Power (MW)", row=1, col=1)
    fig.update_yaxes(title_text="Prices ($/MWh)", row=2, col=1)

    h_per_interval = region_metadata[region]["h_per_interval"]
    ms_per_interval = h_per_interval * ms_per_h
    fig.update_xaxes(
        title_text="Time", minor=dict(dtick=ms_per_interval, showgrid=True)
    )

    fig.show()

    for ext in ['svg', 'pdf']:
        fig.write_image(os.path.join(results_location, f"plot-{region}.{ext}"))